In [28]:
from pandas.core.interchange.dataframe_protocol import DataFrame

"""
Read in the metadata and merge with the novelty data for lit data
"""

data_dir = '../precocity'
all_dfs = []

import pandas as pd
import os
import re
# Load metadata
# df_meta = pd.read_csv('../metadata/19cfictionmeta.tsv', sep='\t')

df_meta = pd.read_csv('../metadata/enriched_metadata.tsv', sep='\t')


# Load all paper data files
print("Reading data files...")
for file in os.listdir(data_dir):
    if file.endswith('.tsv'):
        match = re.search(r'(\d{4})', os.path.basename(file))
        year = match.group(1) if match else 'unknown'
        # print(f"Processing {csv} (year: {year})")
        path = os.path.join(data_dir, file)
        df = pd.read_csv(path, sep='\t')
        df['BOOK_ID'] = df['docid']
        df['decade'] = year
        merged = df_meta.merge(df, on='BOOK_ID', how='left')
        all_dfs.append(merged)


# gender_meta = pd.read_csv('lit_data/author_genders_lit.csv')
#read in the gender data we have already

# gender_meta = gender_meta[gender_meta['gender'].isin(['male', 'female'])]

df_all = pd.concat(all_dfs, ignore_index=True)


"deduplicate the data by selecting one set of parameters ted recommended"

#filter for only male/female gender and also for the single set of novelty results (time chunks etc)

#chunks_used
    #use 0.25 (top 25% most novel chunks)
df_all = df_all.loc[df_all['chunks_used'] == 0.25]

#time_radius
    #use 20
df_all = df_all.loc[df_all['time_radius'] == 20]


#filtered
    #use trainauthquote
df_all = df_all.loc[df_all['filtered'] == 'trainauthquote']



#fraction_compared
    #10 percent most similar vs all articles... use all articles aka 1.0

df_all = df_all.loc[df_all['fraction_compared'] == 1.0]





Reading data files...


In [ ]:
#where is the original gender data
df_meta = pd.read_csv('../metadata/19cfictionmeta.tsv', sep='\t')


In [29]:
# df_all.to_csv('df_all_fiction.csv')

In [30]:
df_meta.head()

,BOOK_ID,FILENAME,LIBRARIES,TITLE,AUTH_LAST,AUTH_FIRST,AUTH_ID,WRITTEN_AS,PUBL_CITY,PUBLISHER,...,birthyear,author_age,is_bestseller,bestseller_contrast,is_reviewed,reviewed_contrast,is_syllabus,syllabus_contrast,is_discussed,discussed_contrast
0,10517,00010517.txt,49.0,Place in the city,Fast,Howard,A_01139,NaN,New York,"""Harcourt, Brace & Co.""",...,1919.0,18.0,0,0,0,0,0,0,0,0
1,uc1.$b243800,uc1.$b243800.txt,NaN,The trap : a novel,Jacobson,['Dan'],NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0
2,21599,00021599.txt,331.0,Umbertina,Barolini,Helen,A_00173,NaN,New York,Seaview Books,...,1925.0,54.0,0,0,0,0,0,0,1,0
3,uc1.b4372570,uc1.b4372570.txt,NaN,The Odyssey of a nice girl,Suckow,['Ruth'],NaN,NaN,NaN,NaN,...,1892.0,33.0,0,0,0,0,0,0,0,0
4,loc.ark+=13960=t98636g92,loc.ark+=13960=t98636g92.txt,NaN,Granville;,Guttersen,['Granville'],NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


In [31]:
def clean_author_name(name):
    if isinstance(name, str):
        name = name.strip()
        name = name.replace('[', '').replace(']', '')
        name = name.replace('"', '').replace("'", '')
        return name.strip()
    return name

In [32]:
df_all['author'] = df_all['AUTH_FIRST'] + df_all['AUTH_LAST']

In [33]:
df_all.to_csv('df_all_fiction.csv')


In [34]:
df_all['author'] = df_all['author'].apply(clean_author_name)


In [35]:
unique_authors = df_all['author'].unique()

In [36]:
len(unique_authors)

2773

In [37]:
df_gender = pd.DataFrame(columns=['author','gender'])

df_gender['author'] = unique_authors


In [38]:
# df_gender['author'] = df_gender['author'].str.split(',').apply(lambda x: x[1].strip() + ' ' + x[0].strip())


In [39]:
def flip_name(n):
    if ',' in n:
        last, first = n.split(',', 1)
        return f"{first.strip()} {last.strip()}"
    return n  # leave unchanged

df_gender['author'] = df_gender['author'].apply(flip_name)


TypeError: argument of type 'float' is not iterable

In [ ]:
#add gender metadata

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import time

# Example DataFrame
# df = pd.DataFrame({
#     'author': ['Mary Shelley', 'H.G. Wells', 'J.K. Rowling'],
#     'title': ['Frankenstein', 'The Time Machine', 'Harry Potter']
# })

# Wikidata SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

def get_gender_wikidata(name):
    """
    Query Wikidata for a person's gender given their name.
    Returns 'male', 'female', or None.
    """
    query = f"""
    SELECT ?genderLabel WHERE {{
      ?person wdt:P31 wd:Q5;        # instance of human
              rdfs:label "{name}"@en;
              wdt:P21 ?gender.     # property for gender/sex
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        bindings = results['results']['bindings']
        if bindings:
            return bindings[0]['genderLabel']['value']
        else:
            return None
    except Exception as e:
        print(f"Error querying {name}: {e}")
        return None

# Add gender column
genders = []
for author in df_gender['author']:
    gender = get_gender_wikidata(author)
    genders.append(gender)
    time.sleep(1)  # be nice to the server

df_gender['gender'] = genders

# print(df)

In [27]:
df_gender.head()

,author,gender
0,"Ralph, DelahayePaine",NaN
1,FrederickLandis,NaN
2,D. H.Lawrence,NaN
3,Edward MandellHouse,NaN
4,UptonSinclair,NaN


In [26]:
df_gender['gender'].value_counts(dropna=False)


gender
NaN    2773
Name: count, dtype: int64

In [25]:
df_gender[df_gender['gender'].isin(['female','male'])]
# gender_meta = gender_meta[gender_meta['gender'].isin(['male', 'female'])]


,author,gender


In [34]:
df_gender.to_csv('df_gender.csv')